## function calling的講解

函式呼叫使您更容易從生成模型中獲取結構化資料輸出。然後，您可以使用這些輸出來呼叫其他 API 並將相關回應資料傳回給模型。換句話說，函數呼叫可以幫助您將產生模型連接到外部系統，以便產生的內容包含最新、最準確的資訊。

您可以為 Gemini 模型提供功能描述。這些是您使用應用語言編寫的函數（也就是說，它們不是 Google Cloud Functions）。模型可能會要求您呼叫函數並傳回結果以幫助模型處理您的查詢。

### 光線控制的API
想像你有一個基本控制光線的應用程式(api)和您允許使用者使用輸入的文字控制光線的明亮度和溫度.可以使用函數呼叫功能了解使用者改變亮度的要求和轉換它們進入到api.
| Parameter | Type | Required | Description |
|:--|:--|:--|:--|
| brightness | number | yes | Light level from 0 to 100. Zero is off and 100 is full brightness. |
| colorTemperature | string | yes | Color temperature of the light fixture which can be daylight, cool or warm. |

In [ ]:
{
  "brightness": "50",
  "colorTemperature": "daylight"
}

### 定義一個自訂的函式
建立一個符合API需求的函式,這個function一定是被定義在我們應用程式內,function內的程式可以呼叫外部的API.模型不會直接呼叫我們的函式,所以只是註冊一個function.這個function要有很明確的描素

**英文**
- 自訂function的參數和傳出值要描素清楚
- 建立模型時的tools引數值
- enable_automatic_function_calling=True,此設定會由model決定是否執行function



In [2]:
import os
import google.generativeai as genai

def set_light_value(brightness:int, color_temp:str)->str:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness":brightness,
        "colorTemperature":color_temp
    }
genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model = genai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    tools=[set_light_value]
    )
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
response.text

"OK. I've set the lights to 30% brightness and a warm color temperature.\n"

**中文版**

In [ ]:
import os
import google.generativeai as genai

def set_light_value(brightness:int, color_temp:str)->str:
    """設定房間的光線亮度和光線溫度 (模擬的API).

    Parameters:
        brightness: 亮度的等級從0~100,如果為0代表關閉光線,如果為100代表光線全開
        color_temp: 代表光線的溫度,有3個等級 `正常溫度`, `冷溫度` or `溫暖溫度`.

    Returns:
        詞典物件設定光線亮度和光線溫度.
    """
    print(brightness)
    print(color_temp)
    return {
        "brightness":brightness,
        "colorTemperature":color_temp
    }

model = genai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction='''
    1. 如果提出的問題不明確,您無法了解,請再次詢問使用者,並請給予幾個提示範例
    2. 如果提供的問題不是呼叫`set_light_value`函式,告訴使用者提供開燈的問題。
    ''',
    tools=[set_light_value]
    )
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('你是誰?')
response.text 

'我是一個大型語言模型，由 Google 訓練。我能幫助你處理許多類型的問題，包含提供資訊、創作文章，甚至是寫程式碼。\n\n不過，由於我目前只被設定能夠控制房間的光線，如果您有任何關於開燈的需求，請隨時告訴我。例如，您可以問：「將亮度設為 50，並將色溫設為正常溫度」，或者「將燈關掉」。\n'